In [1]:
import numpy as np
import os, sys
from datetime import datetime
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [3]:
# person_id = 2 = beth
prep_id = 'DK52'
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    # bili, do your transformation on x,y,section here
    row_dict[structure] = [row.x, row.y, row.section]


In [ ]:
def brain_to_atlas_transform(brain_coord, r, t):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates. 
    """
    atlas_coord = []
    src_transformed = src @ r + t 
    return atlas_coord


In [4]:
person_id = 28 # bili, this is your database ID
R = np.eye(3)
t = np.array([0,0,0])
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    brain_to_atlas(x,y,z,r,t)
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    #print(k, structure, x, y, section)
    x,y,section = brain_to_atlas_transform((x,y,section), R, t)

    com = CenterOfMass(prep_id=prep_id, structure=structure, x=x, y=y, section=section,
                   created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned')
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()

